<a href="https://colab.research.google.com/github/duper203/upstage_cookbook/blob/main/from_togetherai/Structured_Text_Extraction_from_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Structured Data from Images
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Structured_Text_Extraction_from_Images.ipynb)

## Introduction

In this notebook we will demonstrate how you can use a language vision model(Solar DocVision) along with an LLM that has JSON mode enabled(Solar-Pro) to extract structured text from images.

In our case we will extract line items from an invoice in the form of a JSON.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/structured_text_image.png?raw=1" width="750">


### Install relevant libraries

In [38]:
!pip install -qU openai

In [39]:
import os, json
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

## Create Invoice Structure using Pydantic

We need a way of telling the LLM what structure to organize information into - including what information to expect in the receipt. We will do this using `pydantic` models.

Below we define the required classes.

- Each line item on the receipt will have a `name`, `price` and `quantity`. The `Item` class specifies this.
- Each receipt/invoice is a combination of multiple line `Item` elements along with a `total` price. The `Receipt` class specifies this.

In [40]:
import json
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str
    price: float
    quantity: int = Field(default=1)

class Receipt(BaseModel):
    items: list[Item]
    total: float

## Lets bring in the reciept that we want to extract information from

Notice that this is a real receipt with multiple portions that are not relevant to the line item extraction structure we've outlined above.

<img src="https://ocr.space/Content/Images/receipt-ocr-original.webp" height="500">

## 1. Extract Information Receipt

We will use the Llama 3.2 90B Vision model to extract out information in normal text format.

In [41]:
from openai import OpenAI

getDescriptionPrompt = "Extract out the details from the receipt image. Identify the name, price and quantity of each item. Also specify the total. In json format"

imageUrl = "https://ocr.space/Content/Images/receipt-ocr-original.jpg"

client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"],
    base_url="https://api.upstage.ai/v1/solar"
)

response = client.chat.completions.create(
    model="solar-docvision",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": getDescriptionPrompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],

        }
    ],
)

info=response.choices[0].message.content

In [42]:
print(info)

 {'image_type': 'Table', 'text': '<thead></thead><tbody><tr><td colspan="2">Walmart� Ⓡ</td></tr><tr><td colspan="2">Save money. Live better.</td></tr><tr><td>(330) 3339 MANAGER DIANA EARNEST 231 BLUEBELL DR. SW</td><td>3991</td></tr><tr><td>NEW PHILADELPHIA OH 44663 ST# 02115 OP# 009044 TE# 44 TTR# 01301</td><td></td></tr><tr><td>PEET TOY 004747 514846</td><td>11977 X</td></tr><tr><td>FLOPPY PUPPY 07006060332153</td><td>41977 X</td></tr><tr><td>SSSSUPREME S 084669998003228</td><td>51927 X</td></tr><tr><td>2 MUNCHKY DUMBEL 06881133008796</td><td>31777 X</td></tr><tr><td>DOG TREAT 00711900136654</td><td>21520 X</td></tr><tr><td>PED PCH 0023100011802</td><td>0150 X</td></tr><tr><td>PED PCH 1 0023100011802</td><td>0150 X</td></tr><tr><td>COUPON 23100 0523100037000</td><td>1100-0 X</td></tr><tr><td>HNNYMD SMORR 0884912268837</td><td>31980 O</td></tr><tr><td>FRENCH DERS 004132100655</td><td>11980 N</td></tr><tr><td>3 ORANGES 0014668350001</td><td>5147 N</td></tr><tr><td>BABY CARROT 003338366

Notice that the model is not perfect and wasn't able to extract out some line items. It's hard for most models to perform this zero-shot extraction of data from images. A way to improve this is to finetune the model using [Visual Intruction Tuning](https://arxiv.org/abs/2304.08485).

## 2. Organize Information as JSON

We will use Llama 3.1 70B with structured generation in JSON mode to organize the information extracted by the vision model into an acceptable JSON format that can be parsed.

`Meta-Llama-3.1-70B-Instruct-Turbo` will strcitly respect the JSON schema passed to it.

In [43]:
extract = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "The following is a detailed description of all the items, prices and quantities on a receipt. Extract out information. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": info,
            },
        ],
        model="solar-pro",
        response_format={
            "type": "json_object",
            "schema": Receipt.model_json_schema(),
        },
    )

In [44]:
output = json.loads(extract.choices[0].message.content)
print(json.dumps(output, indent=2))

{
  "items": [
    {
      "description": "Peety Floppy Puppy",
      "quantity": 41977,
      "price": "X"
    },
    {
      "description": "SSSSupreme S",
      "quantity": 51927,
      "price": "X"
    },
    {
      "description": "2 Munchky Dumbel",
      "quantity": 31777,
      "price": "X"
    },
    {
      "description": "Dog Treat",
      "quantity": 21520,
      "price": "X"
    },
    {
      "description": "PED PCH",
      "quantity": 150,
      "price": "X"
    },
    {
      "description": "PED PCH 1",
      "quantity": 150,
      "price": "X"
    },
    {
      "description": "Coupon 23100 0523100037000",
      "quantity": 1100,
      "price": "-0 X"
    },
    {
      "description": "HNNYMD Smorrr",
      "quantity": 31980,
      "price": "O"
    },
    {
      "description": "FRENCH DERS",
      "quantity": 11980,
      "price": "N"
    },
    {
      "description": "3 ORANGES",
      "quantity": 5147,
      "price": "N"
    },
    {
      "description": "BABY CARRO

Althought with some missed line items we were able to extract out structured JSON from an image in a zero shot manner! To improve the results for your pipeline and make them production ready I recommend you [finetune](https://docs.together.ai/docs/fine-tuning-overview) the vision model on your own dataset!